# Import Statements

In [2]:
import pandas as pd
import numpy as np
import sqlite3
from matplotlib import pyplot as plt
import dominate
from dominate.tags import *
from dominate.svg import *
import espn_api.football
from espn_api.football import League, Player
import time
import pickle
import uuid
from typing import Iterable
import math

from python import constants, functions, espn_data

from IPython.core.display import HTML

# Testing and Debugging

In [6]:
seasons = [functions.summary_table(data=constants.GAME_DATA, year=year) for year in constants.YEARS]
data = pd.concat(seasons)
data = data.loc[data['Team'] == 'Andrew']

data

,Week,Year,Team,Wins,Losses,Record,Points For,Points Against,PF/G,PF/G+,PA/G,PA/G+,Avg Margin,Luck Score,Champ Flag,Ranking
7,None,2019,Andrew,6,7,6-7,1574.14,1598.96,121.09,102,123.00,104,-1.91,-7,0,8
4,None,2020,Andrew,6,7,6-7,1645.84,1545.26,126.60,104,118.87,98,7.74,-2,0,5
0,None,2021,Andrew,10,4,10-4,1901.54,1755.20,135.82,110,125.37,101,10.45,-3,1,1
3,None,2022,Andrew,8,6,8-6,1599.88,1650.06,114.28,100,117.86,104,-3.58,-4,0,4
3,None,2023,Andrew,8,6,8-6,1672.62,1532.50,119.47,102,109.46,94,10.01,2,0,4
1,None,2024,Andrew,10,4,10-4,1873.18,1681.98,133.80,113,120.14,102,13.66,-3,0,2


In [ ]:
seasons = [summary_table(data=constants.GAME_DATA, year=year) for year in constants.YEARS]
data = pd.concat(seasons)
data = data.loc[data['Team'] == 'Haris']

averages = {
    'Year':'Total',
    'Ranking':round(data['Ranking'].mean(), 2),
    'Points For':round(data['Points For'].mean(), 2),
    'Points Against':round(data['Points Against'].mean(), 2),
    'PF/G':round(data['PF/G'].mean(), 2),
    'PF/G+':round(data['PF/G+'].mean(), 2),
    'Avg Margin':round(data['Avg Margin'].mean(), 2),
    'Luck Score':round(data['Luck Score'].mean(), 2)
}

,Team,Wins,Losses,Record,Points For,Points Against,PF/G,PF/G+,PA/G,PA/G+,Avg Margin,Luck Score,Champ Flag,Ranking,Year
8,Haris,5,8,5-8,1254.30,1304.58,96.48,97,100.35,101,-3.87,-5,0,9,2018
4,Haris,7,6,7-6,1548.30,1483.26,119.10,101,114.10,96,5.00,1,0,5,2019
9,Haris,3,10,3-10,1440.82,1773.88,110.83,91,136.45,112,-25.62,-13,0,10,2020
2,Haris,8,6,8-6,1855.22,1655.12,132.52,107,118.22,96,14.29,4,0,3,2021
6,Haris,7,7,7-7,1647.48,1579.54,117.68,103,112.82,99,4.85,-3,0,7,2022
0,Haris,12,2,12-2,1801.34,1456.30,128.67,110,104.02,89,24.65,10,1,1,2023
9,Haris,3,11,3-11,1468.60,1616.88,104.90,89,115.49,98,-10.59,1,0,10,2024


In [3]:
year = 2025
week = 2

data = constants.PLAYER_GAME_DATA.loc[(constants.PLAYER_GAME_DATA['Year'] == year) & (constants.PLAYER_GAME_DATA['Week'] == week)].copy()
data['Proj Diff'] = round(data['Points'] - data['Projected Points'], 2)

bench_df = data.loc[data['Slot Position'] == 'BE', ['Team','Player','Position','Points']].copy().sort_values('Points', ascending=False).reset_index(drop=True).head(5)
bench_table = functions.df_to_table(data=bench_df, table_id='topScoringBench')
bench_div = functions.content_container(title='Highest Scoring Bench Players', content=bench_table)

under_df = data.loc[data['Slot Position'] != 'BE', ['Team','Player','Position','Proj Diff']].copy().sort_values('Proj Diff', ascending=True).reset_index(drop=True).head(5)
under_table = functions.df_to_table(data=under_df, table_id='topUnderPerformers')
under_div = functions.content_container(title='Biggest Flops', content=under_table)

over_df = data.loc[data['Slot Position'] != 'BE', ['Team','Player','Position','Proj Diff']].copy().sort_values('Proj Diff', ascending=False).reset_index(drop=True).head(5)
over_table = functions.df_to_table(data=over_df, table_id='topOverPerformers')
over_div = functions.content_container(title='Biggest Successes', content=over_table)


display(HTML(str(bench_div)))
display(HTML(str(under_div)))
display(HTML(str(over_div)))

Team,Player,Position,Points
Ethan,Jared Goff,QB,33.96
Zach,Wan'Dale Robinson,WR,28.2
Andrew,Drake Maye,QB,26.3
Tyler,Jordan Love,QB,20.88
Ethan,Jauan Jennings,WR,19.9


Team,Player,Position,Proj Diff
Ethan,Derrick Henry,RB,-15.61
Kevin,Justin Fields,QB,-15.27
Kevin,Broncos D/ST,D/ST,-12.53
Nathan,Omarion Hampton,RB,-12.22
Dante,Courtland Sutton,WR,-12.13


Team,Player,Position,Proj Diff
Michael,Amon-Ra St. Brown,WR,22.28
Kevin,Malik Nabers,WR,19.37
Carter,Rome Odunze,WR,18.29
Andrew,Ja'Marr Chase,WR,16.64
Kevin,Brandon Aubrey,K,14.66


### Best Lineup table
### Includes Projection Hit/Miss

In [90]:
year = 2025
week = 2

data = constants.PLAYER_GAME_DATA.loc[(constants.PLAYER_GAME_DATA['Year'] == year) & (constants.PLAYER_GAME_DATA['Week'] == week)].copy()
data = data.sort_values('Points', ascending=False)

best_score = []

for team in data['Team'].unique():
    data_team = data.loc[data['Team'] == team].copy()
    actual = data_team.loc[data_team['Slot Position'] != 'BE', 'Points'].sum()
    proj = data_team.loc[data_team['Slot Position'] != 'BE', 'Projected Points'].sum()
    proj_diff = actual - proj

    score = 0
    for position in ['QB','TE','D/ST','K']:
        score += data_team.loc[data_team['Position'] == position, 'Points'].max()

    for position in ['RB','WR']:
        score += data_team.loc[data_team['Position'] == position, 'Points'].nlargest(2).sum()

    third_rb = data_team.loc[data_team['Position'] == 'RB', 'Points'].nlargest(3).iloc[-1]
    third_wr = data_team.loc[data_team['Position'] == 'WR', 'Points'].nlargest(3).iloc[-1]
    second_te = data_team.loc[data_team['Position'] == 'TE', 'Points'].nlargest(2).iloc[-1]

    score += max(third_rb, third_wr, second_te)

    coverage = round(actual/score * 100)

    best_score.append({'Team':team, 'Actual Score':actual, 'Projection Hit/Miss':proj_diff, 'Best Score':score, 'Pct Coverage':f'{coverage}%'})

pd.DataFrame(best_score).sort_values('Pct Coverage', ascending=False)


,Team,Actual Score,Projection Hit/Miss,Best Score,Pct Coverage
4,Carter,158.88,35.54,160.38,99%
1,Kevin,136.08,11.19,146.10,93%
0,Michael,152.92,35.50,167.32,91%
9,Noah,100.34,-20.22,110.04,91%
7,Haris,123.84,5.15,138.94,89%
10,Nathan,97.80,-22.57,113.40,86%
8,Dante,116.00,-10.02,137.20,85%
6,Tyler,116.34,14.43,138.68,84%
2,Andrew,103.92,-15.86,124.90,83%
11,McGwire,107.60,-17.26,130.70,82%


### Score by Position Breakdown

In [93]:
year = 2024
# week = 1

data = constants.PLAYER_GAME_DATA.loc[(constants.PLAYER_GAME_DATA['Year'] == year) & (constants.PLAYER_GAME_DATA['Week'] <= 14)].copy()
data = data.loc[data['Slot Position'] != 'BE']

score_breakdown = []
for team in data['Team'].unique():
    score_line = {'Team':team}
    data_team = data.loc[data['Team'] == team].copy()

    full_score = data_team['Points'].sum()
    score_line['Full Score'] = full_score

    pos_scores = {}
    for position in ['QB','RB','WR','TE','D/ST','K']:
        pos_scores[position] = data_team.loc[data_team['Position'] == position, 'Points'].sum()
        score_line[f'{position} Points'] = pos_scores[position]


    score_breakdown.append(score_line)

pd.DataFrame(score_breakdown)

,Team,Full Score,QB Points,RB Points,WR Points,TE Points,D/ST Points,K Points
0,Kevin,1777.24,248.04,536.30,525.40,173.50,142.0,152.0
1,Ethan,1540.46,247.26,371.40,576.20,119.60,83.0,143.0
2,Zach,1762.04,234.54,556.10,571.68,149.72,135.0,115.0
3,Tyler,1604.46,314.38,450.70,458.58,142.80,87.0,151.0
4,McGwire,1413.62,234.40,331.30,398.22,231.70,116.0,102.0
5,Andrew,1873.18,342.20,502.50,607.48,183.00,130.0,108.0
6,Haris,1468.60,246.20,353.90,483.80,192.70,42.0,150.0
7,Noah,1608.14,286.74,535.90,378.70,150.80,134.0,122.0
8,Carter,1872.08,290.66,561.02,662.00,159.40,91.0,108.0
9,Nathan,1712.72,222.32,557.50,495.80,216.10,100.0,121.0


In [91]:
functions.summary_table(constants.GAME_DATA, year=2024)

,Week,Year,Team,Wins,Losses,Record,Points For,Points Against,PF/G,PF/G+,PA/G,PA/G+,Avg Margin,Luck Score,Champ Flag,Ranking
0,None,2024,Kevin,11,3,11-3,1777.24,1568.64,126.95,108,112.05,95,14.90,5,0,1
1,None,2024,Andrew,10,4,10-4,1873.18,1681.98,133.80,113,120.14,102,13.66,-3,0,2
2,None,2024,Carter,10,4,10-4,1872.08,1654.36,133.72,113,118.17,100,15.55,-4,0,3
3,None,2024,Dante,10,4,10-4,1661.44,1557.08,118.67,100,111.22,94,7.45,8,0,4
4,None,2024,Zach,8,6,8-6,1762.04,1670.20,125.86,107,119.30,101,6.56,-5,1,5
5,None,2024,Noah,8,6,8-6,1608.14,1616.62,114.87,97,115.47,98,-0.61,3,0,6
6,None,2024,Tyler,8,6,8-6,1604.46,1565.22,114.60,97,111.80,95,2.80,6,0,7
7,None,2024,Ethan,7,7,7-7,1540.46,1707.74,110.03,93,121.98,103,-11.95,-1,0,8
8,None,2024,Nathan,5,9,5-9,1712.72,1682.00,122.34,104,120.14,102,2.19,-10,0,9
9,None,2024,Haris,3,11,3-11,1468.60,1616.88,104.90,89,115.49,98,-10.59,1,0,10
